In [ ]:
import socket
import threading


class ChatServer:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.server_socket = None
        self.connected_clients = []
        self.message_buffer = {} # буфер хранит сообщения для id клиентов которые не подкючены

    def start(self):
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) # создание сокета
        self.server_socket.bind((self.host, self.port)) # связываем сокет с портом, где он будет ожидать сообщения
        self.server_socket.listen(5) # указываем сколько может сокет принимать соединений

        print(f"Chat server started on {self.host}:{self.port}")

        while True:
            client_socket, client_address = self.server_socket.accept() # начинаем принимать соединения
            client_id = client_socket.recv(1024).decode() # принимаем данные от клиента, по 1024 байт и преобразование их в строку с помощью decode
            self.connected_clients.append((client_socket, client_id)) # добавляем клиента в список подключенных
            
            # Проверка наличия сообщений в буфере для данного клиента
            if client_id in self.message_buffer:
                for message in self.message_buffer[client_id]:
                    client_socket.send(message.encode())  # Отправка сообщения из буфера клиенту
                del self.message_buffer[client_id]  # Удаление сообщений из буфера после отправки

            client_thread = threading.Thread(target=self.handle_client, args=(client_socket, client_id)) # создание потока для обработки клиента (каждый клиент обрабатывается в отдельном потоке, параллельная обработка сообщений)
            client_thread.start() # запуск потока

    def handle_client(self, client_socket, client_id): # какой клиент (id клиента) подключился
        print(f"Client {client_id} connected")

        while True:
            message = client_socket.recv(1024).decode() # сообщение, полученное от клиента

            if message == '/quit':
                for i in range(len(self.connected_clients)):
                    if client_id == self.connected_clients[i]:
                        self.connected_clients.pop(i) # удаление клиента из подключенных
                break
            else:
                print(f"RECEIVED MESSAGE FROM {client_id}") # подтверждение получения сообщения
                recipient_id, content = message.split(':', 1) # разделение сообщения на id получателя и содержимое (разделение позволяет серверу определить, кому отправить сообщение и что именно отправить)
                self.send_message_to_client(recipient_id, client_id, content) # отправка сообщения

        client_socket.close() # закрытие соединения с клиентом
        print(f"Client {client_id} disconnected")

    def send_message_to_client(self, recipient_id, sender_id, message):
        for client_socket, client_id in self.connected_clients: # присваивает значение client_socket, client_id элементам из пары
            if client_id == recipient_id: # проверка на соответствие
                full_message = f"New message from '{sender_id}':\n'{message}'\n"
                client_socket.send(full_message.encode()) # отправка сообщения клиенту если он доступен
                break
        else:
            # Получатель не найден, добавляем сообщение в буфер
            if recipient_id not in self.message_buffer:
                self.message_buffer[recipient_id] = []
            self.message_buffer[recipient_id].append(f"New message from '{sender_id}':\n'{message}'\n") #вот тут отправим сообщение клиенту которое улетит в буфер


if __name__ == '__main__':
    server = ChatServer('localhost', 55000) # подключаемся к серверному сокету, создание экземпляра сервера
    server.start()

Chat server started on localhost:55000
Client 1 connected
